This notebook mainly focuses on feature selection aspect of the dataset

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sn
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn import model_selection
from sklearn.model_selection import train_test_split

In [ ]:
train_data = pd.read_csv("/kaggle/input/mobile-price-classification/train.csv")

In [ ]:
train_data.head()

In [ ]:
train_data.tail()

In [ ]:
train_data.isnull().any()

In [ ]:
train_data['price_range'].value_counts()

## Univariate Feature Selection

Uses staistical test to select those features that have strongest relationship with target variable.

In [ ]:
x = train_data.iloc[:,0:20]
y = train_data.iloc[:,20:]

In [ ]:
x

In [ ]:
print(x.shape)
print(y.shape)

In [ ]:
best_feat = SelectKBest(chi2,k=10)
fit = best_feat.fit(x,y)

In [ ]:
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(x.columns)

In [ ]:
# concat both dataframes for comparing best scores

score_list = pd.concat([dfcolumns,dfscores],axis=1)
score_list.columns = ["features","scores"]

In [ ]:
score_list

In [ ]:
print(score_list.nlargest(10,"scores"))

Selecting the best features

In [ ]:
train_data.drop(train_data.columns[[1,2,3,5,7,9,10,17,18,19]],axis=1,inplace=True)

In [ ]:
train_data.head()

In [ ]:
train_data.tail()

## Separating dependent and independent variable

In [ ]:
x = train_data.iloc[:,0:10].values
y = train_data.iloc[:,10:].values

In [ ]:
x

In [ ]:
y

## Applying cross validation technique on various models

In [ ]:
# Random Forest Classifier

skfold = StratifiedKFold(n_splits=3,random_state=0)
model_c = RandomForestClassifier(criterion='entropy',n_estimators=10,random_state=0,n_jobs=-1)
results_skfold = model_selection.cross_val_score(model_c, x, y.ravel(), cv=skfold)
print("Accuracy: %.2f%%" % (results_skfold.mean()*100.0))

In [ ]:
# KNN Classifier

skfold = StratifiedKFold(n_splits=3,random_state=0)
model_knn = KNeighborsClassifier(n_neighbors=10)
results_skfold = model_selection.cross_val_score(model_c, x, y.ravel(), cv=skfold)
print("Accuracy: %.2f%%" % (results_skfold.mean()*100.0))

In [ ]:
# Decision Tree Classifier

skfold = StratifiedKFold(n_splits=3,random_state=0)
model_c = DecisionTreeClassifier()
results_skfold = model_selection.cross_val_score(model_c, x, y.ravel(), cv=skfold)
print("Accuracy: %.2f%%" % (results_skfold.mean()*100.0))

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)

In [ ]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=10)

In [ ]:
knn.fit(x_train,y_train.ravel())

## Model

In [ ]:
test_data = pd.read_csv("/kaggle/input/mobile-price-classification/test.csv")

In [ ]:
test_data.head()

In [ ]:
test_data_1 = test_data

In [ ]:
test_data_1 = test_data_1.drop(test_data_1.columns[[0,2,3,4,6,8,10,11,18,19,20]],axis=1)

In [ ]:
predicted_val = knn.predict(test_data_1)

In [ ]:
predicted_val

In [ ]:
test_data['price_range']=predicted_val

In [ ]:
test_data